In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings("ignore")
%matplotlib inline
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [ ]:
#Function Definition
def add_layer(inputs, in_size, out_size, activation_function=None):
  Weights = tf.Variable(tf.random_normal([in_size, out_size])) #in_size - rows  out_size - columns
  biases = tf.Variable(tf.zeros([1, out_size]) + 0.1) #one row
  Wx_plus_b = tf.matmul(inputs, Weights) + biases
  if activation_function is None:
    outputs = Wx_plus_b
  else:
    outputs = activation_function(Wx_plus_b)
  return outputs

In [ ]:
# 1、Load Data and Do Preprocessing
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Machine Learning/Iris_Dataset.csv')
dataset

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
dataset = pd.get_dummies(dataset, columns=['Species']) # One Hot Encoding
dataset

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0
...,...,...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,0,0,1
146,147,6.3,2.5,5.0,1.9,0,0,1
147,148,6.5,3.0,5.2,2.0,0,0,1
148,149,6.2,3.4,5.4,2.3,0,0,1


In [ ]:
values = list(dataset.columns.values) # Get the column names into a list

y = dataset[values[-3:]]
y = np.array(y, dtype='float32') # Get labels

X = dataset[values[1:-3]]
X = np.array(X, dtype='float32') # Get features

In [ ]:
# Shuffle Data
indices = np.random.choice(len(X), len(X), replace=False)
X_values = X[indices]
y_values = y[indices]

In [ ]:
# Create A Training Set, A Validation Set And A Testing Set
test_size = 30
validation_size = 20

X_train = X_values[: -(test_size + validation_size)]
y_train = y_values[: -(test_size + validation_size)]

X_validation = X_values[-(test_size + validation_size) : -test_size]
y_validation = y_values[-(test_size + validation_size) : -test_size]

X_test = X_values[-test_size:]
y_test = y_values[-test_size:]

In [ ]:
# 2. Build Artificial Neural Network

#Parameter Settings
interval = 50
epoch = 1000
hidden_layer_nodes = 8

# Initialize Input Layer (placeholders)
xs = tf.placeholder(shape=[None, 4], dtype=tf.float32)
ys = tf.placeholder(shape=[None, 3], dtype=tf.float32)

# Create Hidden Layers
l1 = add_layer(xs, 4, 8, activation_function=tf.nn.relu)

# Create Output Layer
prediction = add_layer(l1, 8, 3, activation_function=tf.nn.softmax)

# Cost Function
loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction + 1e-10), axis=0)) #1e-10 is used to avoid NaN

# Optimizer & Train Step
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

# Global Variables Initializer
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

In [ ]:
### Do Some Testing BEFORE Training The ANN ###
sess.run(prediction, feed_dict={xs:X_test})
pred = tf.argmax(prediction, axis=1)
sess.run(pred, feed_dict={xs:X_test})

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
# 3. Train Artificial Neural Network
for i in range(epoch+1):
  # training
  sess.run(train_step, feed_dict={xs:X_train, ys: y_train})
  if i % interval == 0:
    # to see the improvement
    pred = tf.argmax(prediction, axis=1)
    actual = tf.argmax(y_validation, axis=1)
    correct = tf.cast(tf.equal(pred, actual), dtype=tf.int32)
    correct = tf.reduce_sum(correct)
    correctNum = sess.run(correct, feed_dict={xs:X_validation})
    accuracy = float(correctNum) / float(validation_size)
    print("Epoch %d, loss: %f, evaluation accuracy: %f." % (i, sess.run(loss, feed_dict={xs: X_train, ys: y_train}), accuracy))

Epoch 0, loss: 272.406738, evaluation accuracy: 0.400000.
Epoch 50, loss: 13.633919, evaluation accuracy: 0.900000.
Epoch 100, loss: 8.869444, evaluation accuracy: 0.900000.
Epoch 150, loss: 7.198456, evaluation accuracy: 0.900000.
Epoch 200, loss: 6.236981, evaluation accuracy: 0.900000.
Epoch 250, loss: 5.591774, evaluation accuracy: 0.900000.
Epoch 300, loss: 5.126077, evaluation accuracy: 0.900000.
Epoch 350, loss: 4.775121, evaluation accuracy: 0.900000.
Epoch 400, loss: 4.502705, evaluation accuracy: 0.950000.
Epoch 450, loss: 4.286177, evaluation accuracy: 0.950000.
Epoch 500, loss: 4.110765, evaluation accuracy: 0.950000.
Epoch 550, loss: 3.966145, evaluation accuracy: 0.950000.
Epoch 600, loss: 3.844897, evaluation accuracy: 0.950000.
Epoch 650, loss: 3.742175, evaluation accuracy: 0.950000.
Epoch 700, loss: 3.653367, evaluation accuracy: 0.950000.
Epoch 750, loss: 3.575594, evaluation accuracy: 0.950000.
Epoch 800, loss: 3.506797, evaluation accuracy: 0.950000.
Epoch 850, los

In [ ]:
# 4. Testing
pred = tf.argmax(prediction, axis=1)
actual = tf.argmax(y_test, axis=1)
correct = tf.cast(tf.equal(pred, actual), dtype=tf.int32)
correct = tf.reduce_sum(correct)
correctNum = sess.run(correct, feed_dict={xs:X_test})
accuracy = float(correctNum) / float(test_size)
print("The accuracy of testing set is %f" % accuracy)

The accuracy of testing set is 1.000000
